In [0]:
from pyspark.sql import SparkSession
spark =( SparkSession
    .builder
    .getOrCreate()
    )

In [0]:
file_path = 'retail_data.bronze.bronze_online_retail'

In [0]:
bronze_df = spark.table(file_path)
display(bronze_df)

In [0]:
bronze_df.printSchema()

Remove records without CustomerID

Remove cancelled invoices

Convert InvoiceDate to timestamp

Create total transaction amount

In [0]:
from pyspark.sql.functions import col,expr,round
bronze_df = bronze_df \
    .withColumn('Quantity',round(expr('try_cast(Quantity as double)')).cast('int')) \
    .withColumn('UnitPrice', expr('try_cast(UnitPrice as double)'))
   # .withColumn('UnitPrice',round(expr('try_cast(UnitPrice as double)')).cast('int'))



In [0]:
bronze_df.printSchema()

In [0]:
from pyspark.sql.functions import col, to_timestamp

silver_df = (
    bronze_df
    .filter(col("CustomerID").isNotNull())

    .filter(~col("InvoiceNo").startswith('C'))

    .filter(col("Quantity").isNotNull() & col("UnitPrice").isNotNull())

    .withColumn("InvoiceDate", to_timestamp(col("InvoiceDate")))

    .withColumn("total_amount", col("Quantity") * col("UnitPrice"))
)

display(silver_df)

In [0]:
silver_df.write.format('delta').mode('overwrite').saveAsTable('retail_data.bronze.silver_online_retail')

In [0]:
%sql
select count(*) from retail_data.bronze.silver_online_retail